In [231]:
import pandas as pd
from datetime import datetime
import sqlite3
import json

In [232]:
assinantesAtivos = pd.read_csv("/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/AssinantesAtivos.csv", sep=",") #
assinantesCancelados = pd.read_csv("/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/AssinantesCancelados.csv", sep =',') #
base_gostos = pd.read_csv('/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/RESPOSTAS_TRATADO_FERNANDO.csv', sep=';')#
pagamentos = pd.read_csv('/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/Pagamentos.csv', sep=',') #
campanha_petisco = pd.read_csv('/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/PNRresponses.csv', sep=';')#
campanha_AHT = pd.read_csv("/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/AHTresponses.csv", sep=';')#
unified_costumers = pd.read_csv('/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/Clientes_unificado.csv', sep=',') #
coupons = pd.read_csv("/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/coupons (1).csv", sep=',')

Tratando os gostos dos clientes

In [233]:
base_gostos.isnull().sum()

id                        0
nome                      1
sobrenome                 1
data_nascimento           1
genero                    1
ddd                       1
como_conheceu             3
data_inscricao            3
rede_social               0
email                     3
assinatura                3
id.1                      3
moda                      3
pessoa_artistica          3
maquiagem                 3
brinquedos_ou_jogos       3
esportes                  3
tem_pet                   3
decorar_casa              3
alimento_saudavel         3
gastronomia               3
carro                     3
mae_ou_pai                3
bebida_alcoolica          3
cuidados_pessoais         3
tecnologia                3
influencer                3
interesses                3
consumidor_id             3
renda_id                  3
restricoes_alimentares    3
tem_cupom                 3
objetivo_id               3
id.2                      3
cidade                    5
estado              

In [234]:
base_gostos = base_gostos.dropna()

In [235]:
base_gostos.columns

Index(['id', 'nome', 'sobrenome', 'data_nascimento', 'genero', 'ddd',
       'como_conheceu', 'data_inscricao', 'rede_social', 'email', 'assinatura',
       'id.1', 'moda', 'pessoa_artistica', 'maquiagem', 'brinquedos_ou_jogos',
       'esportes', 'tem_pet', 'decorar_casa', 'alimento_saudavel',
       'gastronomia', 'carro', 'mae_ou_pai', 'bebida_alcoolica',
       'cuidados_pessoais', 'tecnologia', 'influencer', 'interesses',
       'consumidor_id', 'renda_id', 'restricoes_alimentares', 'tem_cupom',
       'objetivo_id', 'id.2', 'cidade', 'estado', 'consumidor_id.1', 'bairro'],
      dtype='object')

In [236]:
del base_gostos['id']
del base_gostos['id.2']
del base_gostos['ddd']
del base_gostos['id.1']
del base_gostos['consumidor_id.1']
del base_gostos['interesses']
del base_gostos['assinatura']
del base_gostos['consumidor_id']


In [237]:
base_gostos.head()

,nome,sobrenome,data_nascimento,genero,como_conheceu,data_inscricao,rede_social,email,moda,pessoa_artistica,...,cuidados_pessoais,tecnologia,influencer,renda_id,restricoes_alimentares,tem_cupom,objetivo_id,cidade,estado,bairro
0,Eduarda,Costa,12/05/2024,Feminino,-,2024-04-29 13:25:00.000 -0300,0,mariaeduardacostagoncalves8@gmail.com,False,True,...,True,True,False,9.0,"{""restricoes"": [8]}",False,1.0,Itaquaquecetuba,São Paulo,Jardim Nossa Senhora DAjuda
1,Olivia,Carvalho,23/04/2005,Feminino,-,2024-04-29 13:25:00.000 -0300,0,oliviaandradecontato@gmail.com,True,True,...,True,True,True,6.0,"{""restricoes"": [8]}",False,1.0,Caraguatatuba,São Paulo,Loteamento Jardim Aruan
2,Aline,Marchesotti,27/03/1990,Feminino,-,2024-04-29 13:38:00.000 -0300,1,aline.marchesotti@hotmail.com,True,True,...,True,True,False,7.0,"{""restricoes"": [6]}",False,1.0,Campinas,São Paulo,Vila Costa e Silva
3,Amanda,Pinheiro,11/11/1985,Feminino,-,2024-04-29 13:45:00.000 -0300,0,mandinhairon@gmail.com,True,True,...,True,True,True,8.0,"{""restricoes"": [8]}",False,1.0,Guarulhos,São Paulo,Vila Santo Antonio
4,Fabiana,Gomes B Chaves,08/11/1975,Feminino,-,2024-04-29 14:24:00.000 -0300,1,fotografiabiagomes@gmail.com,True,True,...,True,False,True,7.0,"{""restricoes"": [8]}",False,2.0,São Paulo,São Paulo,Chácara Inglesa


In [238]:
restricoes_map = {
    1: "Lactose",
    2: "Glúten",
    3: "Soja",
    4: "Ovo",
    5: "Amendoim",
    6: "Vegetarianismo",
    7: "Veganismo",
    8: "Não tenho restrição",
    9: "Outro"
}

objetivo_map = {
    1: "Quero dar a minha opinião sobre produtos",
    2: "Quero descobrir novos produtos para comprar"
}

faixa_renda_map = {
    6: "R$0 - 3.000",
    7: "R$3.000 - 8.000",
    8: "R$8.000 - 16.000",
    9: "R$16.000 - 24.000",
    10: "Superior a R$24.000"
}

# Transformação da coluna 'restricoes_alimentares' usando JSON e o mapeamento
base_gostos['restricoes_alimentares'] = base_gostos['restricoes_alimentares'].apply(
    lambda x: ", ".join([restricoes_map[id_] for id_ in json.loads(x)["restricoes"]]) 
    if pd.notnull(x) and x.startswith("{") else x
)

# Transformação das colunas 'objetivo' e 'faixa_renda' usando o .loc
base_gostos.loc[base_gostos['objetivo_id'].notnull(), 'objetivo_id'] = base_gostos['objetivo_id'].map(objetivo_map)
base_gostos.loc[base_gostos['renda_id'].notnull(), 'renda_id'] = base_gostos['renda_id'].map(faixa_renda_map)


/var/folders/cl/r3_tlbnx4rj_mp_b2b1gc4f40000gn/T/ipykernel_22061/3842484211.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Quero dar a minha opinião sobre produtos'
 'Quero dar a minha opinião sobre produtos'
 'Quero dar a minha opinião sobre produtos' ...
 'Quero descobrir novos produtos para comprar'
 'Quero dar a minha opinião sobre produtos'
 'Quero dar a minha opinião sobre produtos']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  base_gostos.loc[base_gostos['objetivo_id'].notnull(), 'objetivo_id'] = base_gostos['objetivo_id'].map(objetivo_map)
/var/folders/cl/r3_tlbnx4rj_mp_b2b1gc4f40000gn/T/ipykernel_22061/3842484211.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['R$16.000 - 24.000' 'R$0 - 3.000' 'R$3.000 - 8.000' ...
 'R$8.000 - 16.000' 'R$8.000 - 16.000' 

In [239]:
base_gostos['data_nascimento'] = pd.to_datetime(base_gostos['data_nascimento'], format='%d/%m/%Y', errors='coerce')

# Calcular a idade com base na data atual
today = datetime.now()
base_gostos['idade'] =  base_gostos['data_nascimento'].apply(lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day)) if pd.notnull(x) else None)

In [240]:
bins = [0, 19, 59, 120]  # Limites de idade para Jovens, Adultos e Idosos
labels = ["Jovens", "Adultos", "Idosos"]  # Rótulos para as faixas etárias

# Criar a coluna 'faixa_etaria' com as faixas etárias
base_gostos['faixa_etaria'] = pd.cut(base_gostos['idade'], bins=bins, labels=labels, right=True)

In [241]:
# Manter apenas as colunas necessárias para identificar assinantes
assinantesAtivos.rename(columns={'Customer Email': 'email'}, inplace=True)
assinantesAtivos = assinantesAtivos[['email', 'Status']]
assinantesAtivos['assinantes'] = assinantesAtivos['Status'].apply(lambda x: 1 if x == 'active' else 0)
assinantesAtivos = assinantesAtivos[['email', 'assinantes']]
base_nova = pd.merge(base_gostos, assinantesAtivos, on="email", how="left")

In [242]:
base_nova['assinantes'] = base_nova['assinantes'].apply(lambda x: 1 if pd.notnull(x) else 0)

In [243]:
base_nova.columns

Index(['nome', 'sobrenome', 'data_nascimento', 'genero', 'como_conheceu',
       'data_inscricao', 'rede_social', 'email', 'moda', 'pessoa_artistica',
       'maquiagem', 'brinquedos_ou_jogos', 'esportes', 'tem_pet',
       'decorar_casa', 'alimento_saudavel', 'gastronomia', 'carro',
       'mae_ou_pai', 'bebida_alcoolica', 'cuidados_pessoais', 'tecnologia',
       'influencer', 'renda_id', 'restricoes_alimentares', 'tem_cupom',
       'objetivo_id', 'cidade', 'estado', 'bairro', 'idade', 'faixa_etaria',
       'assinantes'],
      dtype='object')

Tratamento dos assinantes

In [244]:
assinantesAtivos = pd.read_csv("/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/DadosExperimentai/AssinantesAtivos.csv", sep=",")
assinantesAtivos.columns

Index(['Customer ID', 'Customer Email', 'Plan', 'Quantity', 'Interval',
       'Amount', 'Status', 'Created (UTC)', 'Start Date (UTC)',
       'Current Period Start (UTC)', 'Current Period End (UTC)', 'Coupon',
       'Tax Percent', 'Canceled At (UTC)'],
      dtype='object')

In [245]:
assinantesAtivos.head()

,Customer ID,Customer Email,Plan,Quantity,Interval,Amount,Status,Created (UTC),Start Date (UTC),Current Period Start (UTC),Current Period End (UTC),Coupon,Tax Percent,Canceled At (UTC)
0,cus_RC1UzB1huhrZ4X,lionbranco@gmail.com,price_1QAG2iHsrWBJzFX2pWrky42a,1,month,59.9,active,2024-11-10 15:52,2024-11-10 15:52,2024-11-10 15:52,2024-12-10 15:52,NaN,NaN,NaN
1,cus_RC0ijbOaql9B2c,andressa21a@live.com,price_1QAG2iHsrWBJzFX2pWrky42a,1,month,59.9,active,2024-11-10 15:05,2024-11-10 15:05,2024-11-10 15:05,2024-12-10 15:05,NaN,NaN,NaN
2,cus_RBzVjVhlJtx5WQ,natholiveirar03@gmail.com,price_1QAG2iHsrWBJzFX2pWrky42a,1,month,59.9,active,2024-11-10 13:49,2024-11-10 13:49,2024-11-10 13:49,2024-12-10 13:49,NaN,NaN,NaN
3,cus_RBwHefgWZtgleQ,sabrinascandolararp@gmail.com,price_1QAG2iHsrWBJzFX2pWrky42a,1,month,59.9,active,2024-11-10 10:30,2024-11-10 10:30,2024-11-10 10:30,2024-12-10 10:30,NaN,NaN,NaN
4,cus_RBqYJB57nElUiu,joyce_p_almeida@hotmail.com,price_1QAG2iHsrWBJzFX2pWrky42a,1,month,59.9,active,2024-11-10 04:34,2024-11-10 04:34,2024-11-10 04:34,2024-12-10 04:34,NaN,NaN,NaN


In [246]:
del assinantesAtivos['Tax Percent']

In [247]:
assinantesAtivos['Coupon'] = assinantesAtivos['Coupon'].fillna('-')
assinantesAtivos['Canceled At (UTC)']=assinantesAtivos['Canceled At (UTC)'].fillna('N/a')


In [248]:
assinantesAtivos.columns

Index(['Customer ID', 'Customer Email', 'Plan', 'Quantity', 'Interval',
       'Amount', 'Status', 'Created (UTC)', 'Start Date (UTC)',
       'Current Period Start (UTC)', 'Current Period End (UTC)', 'Coupon',
       'Canceled At (UTC)'],
      dtype='object')

In [249]:
assinantesCancelados.columns

Index(['Customer ID', 'Customer Email', 'Plan', 'Quantity', 'Interval',
       'Amount', 'Status', 'Created (UTC)', 'Start Date (UTC)',
       'Current Period Start (UTC)', 'Current Period End (UTC)', 'Coupon',
       'Tax Percent', 'Canceled At (UTC)', 'Ended At (UTC)', 'Customer Name'],
      dtype='object')

In [250]:
del assinantesCancelados['Customer Name']
del assinantesCancelados['Tax Percent']

In [251]:
assinantesTotal = pd.concat([assinantesAtivos, assinantesCancelados], ignore_index=True)

In [252]:
assinantesTotal.columns

Index(['Customer ID', 'Customer Email', 'Plan', 'Quantity', 'Interval',
       'Amount', 'Status', 'Created (UTC)', 'Start Date (UTC)',
       'Current Period Start (UTC)', 'Current Period End (UTC)', 'Coupon',
       'Canceled At (UTC)', 'Ended At (UTC)'],
      dtype='object')

Tratamento da base de Pagamentos

In [253]:
pagamentos.columns

Index(['Created date (UTC)', 'Amount', 'Amount Refunded', 'Decline Reason',
       'Description', 'Refunded date (UTC)', 'Statement Descriptor', 'Status',
       'Seller Message', 'Taxes On Fee', 'Card ID', 'Customer ID',
       'Shipping Address City', 'Shipping Address State', 'Invoice ID'],
      dtype='object')

In [254]:
pagamentos.head(5)

,Created date (UTC),Amount,Amount Refunded,Decline Reason,Description,Refunded date (UTC),Statement Descriptor,Status,Seller Message,Taxes On Fee,Card ID,Customer ID,Shipping Address City,Shipping Address State,Invoice ID
0,2024-11-10 19:37:12,49.0,0.0,NaN,Subscription update,NaN,EXPERIMENTAI,Paid,Payment complete.,0.0,pm_1PxYwaHsrWBJzFX21F322Mdg,cus_QpDMIwnCIbNe0W,São Paulo,SP,in_1QJg0NHsrWBJzFX23wMLF0WJ
1,2024-11-10 19:21:04,49.0,0.0,transaction_not_allowed,Subscription update,NaN,EXPERIMENTAI,Failed,The bank returned the decline code `transactio...,0.0,pm_1Q8QxQHsrWBJzFX2CiFGK7MA,cus_R0RriBqyRRbbzU,Rio de Janeiro,RJ,in_1QJfk1HsrWBJzFX2TjmDMtMq
2,2024-11-10 18:55:01,49.0,0.0,NaN,Subscription update,NaN,EXPERIMENTAI,Paid,Payment complete.,0.0,pm_1QJgHTHsrWBJzFX2E6oeElz8,cus_R03BoF4GDm57su,Vitória,ES,in_1QJHrAHsrWBJzFX2vJE6m8UI
3,2024-11-10 18:41:26,49.0,0.0,NaN,Subscription update,NaN,EXPERIMENTAI,Paid,Payment complete.,0.0,pm_1QJg4fHsrWBJzFX2srLKP5AX,cus_Qy8mvAX7Nrrdfx,Gov. Valadares,MG,in_1QHRHzHsrWBJzFX2K8cyiYRh
4,2024-11-10 18:34:13,32.0,0.0,NaN,Subscription update,NaN,EXPERIMENTAI,Paid,Payment complete.,0.0,pm_1PQC5WHsrWBJzFX2LFAJIcHW,cus_QGjYBIgunKs57j,MairiporÃ,SP,in_1QJf1KHsrWBJzFX2xrVxqvuL


In [255]:
pagamentos['Decline Reason'] = pagamentos['Decline Reason'].fillna('-')
pagamentos['Refunded date (UTC)'] =pagamentos['Refunded date (UTC)'].fillna("N/a")
pagamentos['Amount Refunded']=pagamentos['Amount Refunded'].fillna('-')
pagamentos['Statement Descriptor']=pagamentos['Statement Descriptor'].fillna("-")
pagamentos['Seller Message']=pagamentos['Seller Message'].fillna('-')

In [256]:
del pagamentos['Taxes On Fee']
del pagamentos['Card ID']
del pagamentos['Customer ID']
del pagamentos['Invoice ID']

In [257]:
pagamentos.isnull().sum()

Created date (UTC)         0
Amount                     0
Amount Refunded            0
Decline Reason             0
Description                6
Refunded date (UTC)        0
Statement Descriptor       0
Status                     0
Seller Message             0
Shipping Address City     10
Shipping Address State    11
dtype: int64

In [258]:
pagamentos = pagamentos.dropna()

Tratamento das bases de campanha

In [259]:
campanha_AHT.head(5)

,já_conhecia,nota,intencao_compra,onde_gcomprar,quanto_pagaria,trocar_bebida_por_aht,recomendar_aht,nome,email,idade,genero,renda,bebida,gastronomia,moda,maquiagem,esporte
0,1,5,Certamente compraria,No mercado e na cunha gago,10,O sabor leve e não ter gás,10,primeira dama da experimentai,flaviaferraresi3003@gmail.com,20,Feminino,'+ R$24.000,1,0,1,1,1
1,0,5,Certamente compraria,Nos supermercados,7,Maior facilidade no acesso,9,Gabriel,bieldurigan@gmail.com,22,Masculino,R$8.000 - 16.000,1,1,0,0,1
2,1,5,Certamente compraria,Mercado,7,Vodka,10,Giovanna,giovannamaruyama22@gmail.com,20,Feminino,R$16.000 - 24.000,0,0,0,0,1
3,0,3,"Talvez comprasse, talvez não comprasse",Nao,6,Nada,4,Rafaela,Oliviabrait@gmail.com,20,Feminino,R$0 - 3.000,0,0,0,1,0
4,0,4,Certamente não compraria,No mercado e no oxxo,5,"Nada, nao bebo",6,Pedro mano,Pedropacmano@gmail.com,22,Masculino,R$0 - 3.000,0,1,0,0,1


In [260]:
campanha_AHT.isnull().sum()

já_conhecia              0
nota                     0
intencao_compra          0
onde_gcomprar            0
quanto_pagaria           0
trocar_bebida_por_aht    0
recomendar_aht           0
nome                     0
email                    0
idade                    0
genero                   0
renda                    0
bebida                   0
gastronomia              0
moda                     0
maquiagem                0
esporte                  0
dtype: int64

In [261]:
campanha_petisco.columns

Index(['#', 'Seu pet, que provou o snack, é um cão ou um gato?', 'Other',
       'Você já conhecia esse produto?',
       'Você já conhecia a loja Pet na Rotina?',
       'Como você descreveria a reação do seu pet ao experimentar o petisco?',
       'Você achou o tamanho do petisco adequado para seu pet?',
       'Qual é o principal motivo pelo qual você dá petiscos ao seu pet?',
       'Qual é a sua principal fonte de informação sobre novos produtos para pets?',
       'O que mais te atrai em uma promoção de produtos para pets?',
       'Como você avalia a relação custo-benefício dos produtos que você encontra na Pet na Rotina?',
       'Onde você normalmente compra petiscos para seu pet?',
       'Quando você compra online, qual é o fator mais importante para você?',
       'Quanto você costuma gastar por mês em petiscos para seu pet?',
       'Você está disposto a pagar mais por produtos de alta qualidade e ingredientes naturais?',
       'Qual é o principal motivo que faria você mu

In [262]:
campanha_petisco.head(5)

,#,"Seu pet, que provou o snack, é um cão ou um gato?",Other,Você já conhecia esse produto?,Você já conhecia a loja Pet na Rotina?,Como você descreveria a reação do seu pet ao experimentar o petisco?,Você achou o tamanho do petisco adequado para seu pet?,Qual é o principal motivo pelo qual você dá petiscos ao seu pet?,Qual é a sua principal fonte de informação sobre novos produtos para pets?,O que mais te atrai em uma promoção de produtos para pets?,...,"Na sua próxima compra, qual a melhor opção que traduz sua intenção de comprar o produto?","Por favor, informe seu e-mail",Ao prosseguir você afirma que leu e está de acordo com o Aviso de Privacidade e Termos de Uso da Experimentaí.,name,Response Type,Start Date (UTC),Stage Date (UTC),Submit Date (UTC),Network ID,Tags
0,hjcz3rirfvn0udlf1rvhjcz39g4rkglj,Cão,NaN,1,0,Animado,Adequado,Recompensa,Redes sociais,Produtos adicionais grátis,...,Provavelmente compraria,mariina.souza@hotmail.com,Eu li e estou de acordo,xxxxx,completed,2024-10-17 21:32:11,NaN,2024-10-17 21:35:09,1929f4a04f,NaN
1,niydt2h6dpowt8g3hniydt25vcym02pr,Gato,NaN,0,0,Animado,Adequado,Saúde dental,Redes sociais,Produtos adicionais grátis,...,Provavelmente compraria,lais10cristina@hotmail.com,Eu li e estou de acordo,xxxxx,completed,2024-10-16 14:38:57,NaN,2024-10-16 14:41:10,2eb1289745,NaN
2,h9mylmpv08vet5uwy20aawh9mylmpvct,Cão,NaN,0,0,Muito animado,Adequado,Entretenimento,Sites de reviews,Desconto no preço,...,"Talvez comprasse, talvez não",brunac.villela@gmail.com,Eu li e estou de acordo,xxxxx,completed,2024-10-14 01:57:20,NaN,2024-10-14 02:11:57,ebca20212f,NaN
3,0m9nuyiyxpdouky3c0m9ncyjuwsi6l8k,Cão,NaN,0,0,Muito animado,Adequado,Entretenimento,Redes sociais,Produtos adicionais grátis,...,"Talvez comprasse, talvez não",nutricarolguimaraees@gmail.com,Eu li e estou de acordo,xxxxx,completed,2024-10-14 00:48:31,NaN,2024-10-14 00:50:54,d1328cad9f,NaN
4,nx01fnxckmps4mld3x2hnx01fnx29aom,Gato,NaN,0,0,Indiferente,Muito grande,Recompensa,Redes sociais,Desconto no preço,...,"Talvez comprasse, talvez não",thami.vet@gmail.com,Eu li e estou de acordo,xxxxx,completed,2024-10-13 23:41:50,NaN,2024-10-13 23:44:20,c5231d304f,NaN


In [263]:
# Dicionário para renomear as colunas
colunas_renomear = {
    '#': 'id',
    'Seu pet, que provou o snack, é um cão ou um gato?': 'tipo_pet',
    'Você já conhecia esse produto?': 'conhecia_produto',
    'Você já conhecia a loja Pet na Rotina?': 'conhecia_loja',
    'Como você descreveria a reação do seu pet ao experimentar o petisco?': 'reacao_petisco',
    'Você achou o tamanho do petisco adequado para seu pet?': 'tamanho_adequado',
    'Qual é o principal motivo pelo qual você dá petiscos ao seu pet?': 'motivo_petiscos',
    'Qual é a sua principal fonte de informação sobre novos produtos para pets?': 'fonte_informacao',
    'O que mais te atrai em uma promoção de produtos para pets?': 'atracao_promocao',
    'Como você avalia a relação custo-benefício dos produtos que você encontra na Pet na Rotina?': 'avaliacao_custo_beneficio',
    'Onde você normalmente compra petiscos para seu pet?': 'local_compra',
    'Quando você compra online, qual é o fator mais importante para você?': 'fator_importante_online',
    'Quanto você costuma gastar por mês em petiscos para seu pet?': 'gasto_mensal_petiscos',
    'Você está disposto a pagar mais por produtos de alta qualidade e ingredientes naturais?': 'disposicao_pagar_mais',
    'Qual é o principal motivo que faria você mudar de fornecedor de petiscos e brinquedos para seu cachorro?': 'motivo_troca_fornecedor',
    'Considerando o preço de R$ 28,47 por um pacote de 60g, você compraria esse produto?': 'compraria_produto',
    'Com que frequência você costuma dar petiscos ao seu pet?': 'frequencia_petiscos',
    'Quais são os fatores mais importantes para você ao escolher um petisco?': 'fatores_importantes_petisco',
    'Na sua próxima compra, qual a melhor opção que traduz sua intenção de comprar o produto?': 'intencao_compra'
}

# Renomear as colunas
campanha_petisco.rename(columns=colunas_renomear, inplace=True)

# Colunas a serem removidas
colunas_remover = [
    'Other',
    'Por favor, informe seu e-mail',
    'Ao prosseguir você afirma que leu e está de acordo com o Aviso de Privacidade e Termos de Uso da Experimentaí.',
    'name',
    'Response Type',
    'Start Date (UTC)',
    'Stage Date (UTC)'
]

# Remover as colunas desnecessárias
campanha_petisco.drop(columns=colunas_remover, inplace=True, errors='ignore')


tratamento de unified_costumers

In [264]:
unified_costumers.isnull().sum()

id                         2
Description             1607
Email                      0
Created (UTC)              0
Plan                     485
Status                   485
Cancel At Period End     485
Total Spend                0
Payment Count              0
Refunded Volume            0
Dispute Losses             0
dtype: int64

In [265]:
contagem_payment = unified_costumers.value_counts('Payment Count')
contagem_dispute = unified_costumers.value_counts('Dispute Losses')
contagem_ref = unified_costumers.value_counts('Refunded Volume')

In [266]:

excluir_linhas=unified_costumers[unified_costumers["Dispute Losses"]!= 0].index
unified_costumers=unified_costumers.drop(excluir_linhas)

In [267]:
del unified_costumers['Description']
del unified_costumers['id']
del unified_costumers['Dispute Losses']

In [268]:
unified_costumers = unified_costumers.dropna()

In [269]:
################################################################################################################################################

In [270]:
assinantesTotal.columns

Index(['Customer ID', 'Customer Email', 'Plan', 'Quantity', 'Interval',
       'Amount', 'Status', 'Created (UTC)', 'Start Date (UTC)',
       'Current Period Start (UTC)', 'Current Period End (UTC)', 'Coupon',
       'Canceled At (UTC)', 'Ended At (UTC)'],
      dtype='object')

In [271]:
base_nova.columns

Index(['nome', 'sobrenome', 'data_nascimento', 'genero', 'como_conheceu',
       'data_inscricao', 'rede_social', 'email', 'moda', 'pessoa_artistica',
       'maquiagem', 'brinquedos_ou_jogos', 'esportes', 'tem_pet',
       'decorar_casa', 'alimento_saudavel', 'gastronomia', 'carro',
       'mae_ou_pai', 'bebida_alcoolica', 'cuidados_pessoais', 'tecnologia',
       'influencer', 'renda_id', 'restricoes_alimentares', 'tem_cupom',
       'objetivo_id', 'cidade', 'estado', 'bairro', 'idade', 'faixa_etaria',
       'assinantes'],
      dtype='object')

In [272]:
campanha_AHT.columns

Index(['já_conhecia', 'nota', 'intencao_compra', 'onde_gcomprar',
       'quanto_pagaria', 'trocar_bebida_por_aht', 'recomendar_aht', 'nome',
       'email', 'idade', 'genero', 'renda', 'bebida', 'gastronomia', 'moda',
       'maquiagem', 'esporte'],
      dtype='object')

In [273]:
campanha_petisco.columns

Index(['id', 'tipo_pet', 'conhecia_produto', 'conhecia_loja', 'reacao_petisco',
       'tamanho_adequado', 'motivo_petiscos', 'fonte_informacao',
       'atracao_promocao', 'avaliacao_custo_beneficio', 'local_compra',
       'fator_importante_online', 'gasto_mensal_petiscos',
       'disposicao_pagar_mais', 'motivo_troca_fornecedor', 'compraria_produto',
       'frequencia_petiscos', 'fatores_importantes_petisco', 'intencao_compra',
       'Submit Date (UTC)', 'Network ID', 'Tags'],
      dtype='object')

In [274]:
unified_costumers.columns

Index(['Email', 'Created (UTC)', 'Plan', 'Status', 'Cancel At Period End',
       'Total Spend', 'Payment Count', 'Refunded Volume'],
      dtype='object')

In [275]:
pagamentos.columns

Index(['Created date (UTC)', 'Amount', 'Amount Refunded', 'Decline Reason',
       'Description', 'Refunded date (UTC)', 'Statement Descriptor', 'Status',
       'Seller Message', 'Shipping Address City', 'Shipping Address State'],
      dtype='object')

In [276]:
unified_costumers.columns

Index(['Email', 'Created (UTC)', 'Plan', 'Status', 'Cancel At Period End',
       'Total Spend', 'Payment Count', 'Refunded Volume'],
      dtype='object')

In [277]:
coupons.columns

Index(['id', 'Created (UTC)', 'Valid', 'Duration', 'Duration In Months',
       'Percent Off', 'Amount Off', 'Currency', 'Redeem By (UTC)',
       'Max Redemptions', 'Times Redeemed'],
      dtype='object')

In [279]:
dim_demografico_assinantes = assinantesTotal[[
    'Customer ID', 'Customer Email', 'Plan', 'Quantity', 'Interval',
    'Start Date (UTC)', 'Status'
]]
dim_demografico_assinantes['id_demografico'] = range(1, len(dim_demografico_assinantes) + 1)

fato_assinantes = assinantesTotal[[
    'Customer ID', 'Amount', 'Created (UTC)', 'Current Period Start (UTC)',
    'Current Period End (UTC)', 'Coupon',  'Canceled At (UTC)', 'Ended At (UTC)'
]]
fato_assinantes['id_fato'] = range(1, len(fato_assinantes) + 1)
fato_assinantes = fato_assinantes.merge(dim_demografico_assinantes[['Customer ID', 'id_demografico']], 
                                        on='Customer ID', how='left')

# Subtabelas para base_nova
dim_demografico_base_nova = base_nova[[
    'nome', 'sobrenome', 'data_nascimento', 'genero', 'cidade', 'estado', 'bairro', 'idade', 'faixa_etaria'
]]
dim_demografico_base_nova['id_demografico_base'] = range(1, len(dim_demografico_base_nova) + 1)

dim_gosto_base_nova = base_nova[[
    'moda', 'pessoa_artistica', 'maquiagem', 'brinquedos_ou_jogos', 'esportes', 'tem_pet',
    'decorar_casa', 'alimento_saudavel', 'gastronomia', 'carro', 'bebida_alcoolica',
    'cuidados_pessoais', 'tecnologia', 'mae_ou_pai', 'influencer', 'restricoes_alimentares'
]]
dim_gosto_base_nova['id_gosto_base'] = range(1, len(dim_gosto_base_nova) + 1)

fato_base_nova = base_nova[[
    'data_inscricao', 'rede_social', 'renda_id', 'objetivo_id', 'tem_cupom'
]]
fato_base_nova['id_fato_base'] = range(1, len(fato_base_nova) + 1)

# Subtabelas para campanha_AHT
dim_demografico_aht = campanha_AHT[['nome', 'email', 'idade', 'genero', 'renda']]
dim_demografico_aht['id_demografico_aht'] = range(1, len(dim_demografico_aht) + 1)

dim_gosto_aht = campanha_AHT[['bebida', 'gastronomia', 'moda', 'maquiagem', 'esporte']]
dim_gosto_aht['id_gosto_aht'] = range(1, len(dim_gosto_aht) + 1)

fato_aht = campanha_AHT[[
    'já_conhecia', 'nota', 'intencao_compra', 'onde_gcomprar', 'quanto_pagaria',
    'trocar_bebida_por_aht', 'recomendar_aht'
]]
fato_aht['id_fato_aht'] = range(1, len(fato_aht) + 1)

# Subtabelas para campanha_petisco
dim_demografico_petisco = campanha_petisco[['id', 'tipo_pet']]
dim_demografico_petisco['id_demografico_petisco'] = range(1, len(dim_demografico_petisco) + 1)

fato_petisco = campanha_petisco[[
    'id', 'conhecia_produto', 'tamanho_adequado', 'motivo_petiscos', 'avaliacao_custo_beneficio',
    'gasto_mensal_petiscos', 'disposicao_pagar_mais', 'frequencia_petiscos', 'intencao_compra'
]]
fato_petisco['id_fato_petisco'] = range(1, len(fato_petisco) + 1)

# Subtabelas para unified_costumers
dim_unified_customers = unified_costumers[[
    'Email', 'Plan', 'Status', 'Total Spend', 'Payment Count', 'Refunded Volume'
]]
dim_unified_customers['id_unified'] = range(1, len(dim_unified_customers) + 1)

# Subtabelas para pagamentos
dim_pagamentos = pagamentos[[
    'Created date (UTC)', 'Amount', 'Amount Refunded', 'Status', 'Shipping Address City', 'Shipping Address State'
]]
dim_pagamentos['id_pagamento'] = range(1, len(dim_pagamentos) + 1)

# Subtabelas para coupons
dim_coupons = coupons[['id', 'Valid', 'Duration', 'Percent Off', 'Amount Off']]
dim_coupons['id_cupom'] = range(1, len(dim_coupons) + 1)

/var/folders/cl/r3_tlbnx4rj_mp_b2b1gc4f40000gn/T/ipykernel_22061/184091277.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_demografico_assinantes['id_demografico'] = range(1, len(dim_demografico_assinantes) + 1)
/var/folders/cl/r3_tlbnx4rj_mp_b2b1gc4f40000gn/T/ipykernel_22061/184091277.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fato_assinantes['id_fato'] = range(1, len(fato_assinantes) + 1)
/var/folders/cl/r3_tlbnx4rj_mp_b2b1gc4f40000gn/T/ipykernel_22061/184091277.py:19: SettingWithCopyWar

In [280]:
print(dim_demografico_base_nova.columns)

Index(['nome', 'sobrenome', 'data_nascimento', 'genero', 'cidade', 'estado',
       'bairro', 'idade', 'faixa_etaria', 'id_demografico_base'],
      dtype='object')


In [285]:
import sqlite3
import pandas as pd
# 1. Renomear as colunas dos DataFrames para corresponder aos nomes das colunas nas tabelas do banco de dados

# dim_demografico_assinantes
dim_demografico_assinantes.rename(columns={
    'Customer ID': 'customer_id',
    'Customer Email': 'customer_email',
    'Plan': 'plan',
    'Quantity': 'quantity',
    'Interval': 'interval',
    'Start Date (UTC)': 'start_date_utc',
    'Status': 'status'
}, inplace=True)

# fato_assinantes
fato_assinantes.rename(columns={
    'Customer ID': 'customer_id',
    'Amount': 'amount',
    'Created (UTC)': 'created_utc',
    'Current Period Start (UTC)': 'current_period_start_utc',
    'Current Period End (UTC)': 'current_period_end_utc',
    'Coupon': 'coupon',
    'Tax Percent': 'tax_percent',
    'Canceled At (UTC)': 'canceled_at_utc',
    'Ended At (UTC)': 'ended_at_utc'
}, inplace=True)

# Mesclar para obter id_demografico em fato_assinantes (se ainda não foi feito)
if 'id_demografico' not in fato_assinantes.columns:
    fato_assinantes = fato_assinantes.merge(
        dim_demografico_assinantes[['customer_id', 'id_demografico']],
        on='customer_id', how='left'
    )

# Renomear colunas para outros DataFrames conforme necessário
# Exemplo para dim_unified_customers
dim_unified_customers.rename(columns={
    'Email': 'email',
    'Plan': 'plan',
    'Status': 'status',
    'Total Spend': 'total_spend',
    'Payment Count': 'payment_count',
    'Refunded Volume': 'refunded_volume'
}, inplace=True)

# Renomear colunas para dim_pagamentos
dim_pagamentos.rename(columns={
    'Created date (UTC)': 'created_date_utc',
    'Amount': 'amount',
    'Amount Refunded': 'amount_refunded',
    'Status': 'status',
    'Shipping Address City': 'shipping_address_city',
    'Shipping Address State': 'shipping_address_state'
}, inplace=True)

# Renomear colunas para dim_coupons
dim_coupons.rename(columns={
    'id': 'id_cupom',
    'Valid': 'valid',
    'Duration': 'duration',
    'Percent Off': 'percent_off',
    'Amount Off': 'amount_off'
}, inplace=True)

# 2. Conectar ao banco de dados SQLite
db_path = 'Experimentai.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 3. Desabilitar temporariamente as restrições de chave estrangeira
conn.execute('PRAGMA foreign_keys = OFF;')

# 4. Excluir as tabelas existentes para evitar conflitos
tables = [
    'fato_assinantes',
    'dim_demografico_assinantes',
    'fato_base_nova',
    'dim_gosto_base_nova',
    'dim_demografico_base_nova',
    'fato_aht',
    'dim_gosto_aht',
    'dim_demografico_aht',
    'fato_petisco',
    'dim_demografico_petisco',
    'dim_unified_customers',
    'dim_pagamentos',
    'dim_coupons'
]

for table in tables:
    cursor.execute(f'DROP TABLE IF EXISTS {table};')

conn.commit()

# 5. Criar as tabelas com o esquema adequado

# dim_demografico_assinantes
cursor.execute('''
CREATE TABLE dim_demografico_assinantes (
    id_demografico INTEGER PRIMARY KEY,
    customer_id INTEGER,
    customer_email TEXT,
    plan TEXT,
    quantity INTEGER,
    interval TEXT,
    start_date_utc TEXT,
    status TEXT
);
''')

# fato_assinantes
cursor.execute('''
CREATE TABLE fato_assinantes (
    id_fato INTEGER PRIMARY KEY,
    customer_id INTEGER,
    amount REAL,
    created_utc TEXT,
    current_period_start_utc TEXT,
    current_period_end_utc TEXT,
    coupon TEXT,
    tax_percent REAL,
    canceled_at_utc TEXT,
    ended_at_utc TEXT,
    id_demografico INTEGER,
    FOREIGN KEY(id_demografico) REFERENCES dim_demografico_assinantes(id_demografico)
);
''')

# dim_demografico_base_nova
cursor.execute('''
CREATE TABLE dim_demografico_base_nova (
    id_demografico_base INTEGER PRIMARY KEY,
    nome TEXT,
    sobrenome TEXT,
    data_nascimento TEXT,
    genero TEXT,
    cidade TEXT,
    estado TEXT,
    bairro TEXT,
    idade INTEGER,
    faixa_etaria TEXT
);
''')

# dim_gosto_base_nova
cursor.execute('''
CREATE TABLE dim_gosto_base_nova (
    id_gosto_base INTEGER PRIMARY KEY,
    moda TEXT,
    pessoa_artistica TEXT,
    maquiagem TEXT,
    brinquedos_ou_jogos TEXT,
    esportes TEXT,
    tem_pet TEXT,
    decorar_casa TEXT,
    alimento_saudavel TEXT,
    gastronomia TEXT,
    carro TEXT,
    bebida_alcoolica TEXT,
    cuidados_pessoais TEXT,
    tecnologia TEXT,
    mae_ou_pai TEXT,
    influencer TEXT,
    restricoes_alimentares TEXT
);
''')

# fato_base_nova
cursor.execute('''
CREATE TABLE fato_base_nova (
    id_fato_base INTEGER PRIMARY KEY,
    data_inscricao TEXT,
    rede_social TEXT,
    renda_id INTEGER,
    objetivo_id INTEGER,
    tem_cupom TEXT
);
''')

# dim_demografico_aht
cursor.execute('''
CREATE TABLE dim_demografico_aht (
    id_demografico_aht INTEGER PRIMARY KEY,
    nome TEXT,
    email TEXT,
    idade INTEGER,
    genero TEXT,
    renda TEXT
);
''')

# dim_gosto_aht
cursor.execute('''
CREATE TABLE dim_gosto_aht (
    id_gosto_aht INTEGER PRIMARY KEY,
    bebida TEXT,
    gastronomia TEXT,
    moda TEXT,
    maquiagem TEXT,
    esporte TEXT
);
''')

# fato_aht
cursor.execute('''
CREATE TABLE fato_aht (
    id_fato_aht INTEGER PRIMARY KEY,
    já_conhecia TEXT,
    nota INTEGER,
    intencao_compra TEXT,
    onde_gcomprar TEXT,
    quanto_pagaria REAL,
    trocar_bebida_por_aht TEXT,
    recomendar_aht TEXT
);
''')

# dim_demografico_petisco
cursor.execute('''
CREATE TABLE dim_demografico_petisco (
    id_demografico_petisco INTEGER PRIMARY KEY,
    id INTEGER,
    tipo_pet TEXT
);
''')

# fato_petisco
cursor.execute('''
CREATE TABLE fato_petisco (
    id_fato_petisco INTEGER PRIMARY KEY,
    id INTEGER,
    conhecia_produto TEXT,
    tamanho_adequado TEXT,
    motivo_petiscos TEXT,
    avaliacao_custo_beneficio TEXT,
    gasto_mensal_petiscos REAL,
    disposicao_pagar_mais TEXT,
    frequencia_petiscos TEXT,
    intencao_compra TEXT
);
''')

# dim_unified_customers
cursor.execute('''
CREATE TABLE dim_unified_customers (
    id_unified INTEGER PRIMARY KEY,
    email TEXT,
    plan TEXT,
    status TEXT,
    total_spend REAL,
    payment_count INTEGER,
    refunded_volume REAL
);
''')

# dim_pagamentos
cursor.execute('''
CREATE TABLE dim_pagamentos (
    id_pagamento INTEGER PRIMARY KEY,
    created_date_utc TEXT,
    amount REAL,
    amount_refunded REAL,
    status TEXT,
    shipping_address_city TEXT,
    shipping_address_state TEXT
);
''')

# dim_coupons
cursor.execute('''
CREATE TABLE dim_coupons (
    id_cupom INTEGER PRIMARY KEY,
    valid TEXT,
    duration TEXT,
    percent_off REAL,
    amount_off REAL
);
''')

# 6. Habilitar novamente as restrições de chave estrangeira
conn.execute('PRAGMA foreign_keys = ON;')
conn.commit()

# 7. Inserir os dados dos DataFrames nas tabelas

# Função para verificar e ajustar as colunas antes da inserção (garantir que as colunas correspondam)
def adjust_columns(df, table_name):
    table_info = cursor.execute(f"PRAGMA table_info({table_name});").fetchall()
    table_columns = [col[1] for col in table_info]
    df_columns = df.columns.tolist()
    common_columns = [col for col in df_columns if col in table_columns]
    return df[common_columns]

# Função para inserir DataFrames no banco de dados
def insert_data(df, table_name):
    df = adjust_columns(df, table_name)
    df.to_sql(table_name, conn, if_exists='append', index=False)

# Inserção das tabelas no banco

# dim_demografico_assinantes
insert_data(dim_demografico_assinantes, 'dim_demografico_assinantes')

# fato_assinantes
insert_data(fato_assinantes, 'fato_assinantes')

# dim_demografico_base_nova
insert_data(dim_demografico_base_nova, 'dim_demografico_base_nova')

# dim_gosto_base_nova
insert_data(dim_gosto_base_nova, 'dim_gosto_base_nova')

# fato_base_nova
insert_data(fato_base_nova, 'fato_base_nova')

# dim_demografico_aht
insert_data(dim_demografico_aht, 'dim_demografico_aht')

# dim_gosto_aht
insert_data(dim_gosto_aht, 'dim_gosto_aht')

# fato_aht
insert_data(fato_aht, 'fato_aht')

# dim_demografico_petisco
insert_data(dim_demografico_petisco, 'dim_demografico_petisco')

# fato_petisco
insert_data(fato_petisco, 'fato_petisco')

# dim_unified_customers
insert_data(dim_unified_customers, 'dim_unified_customers')

# dim_pagamentos
insert_data(dim_pagamentos, 'dim_pagamentos')

# dim_coupons
insert_data(dim_coupons, 'dim_coupons')

# 8. Fechar a conexão com o banco
conn.commit()
conn.close()

print("Todas as tabelas foram criadas e os dados foram inseridos com sucesso!")


Todas as tabelas foram criadas e os dados foram inseridos com sucesso!


/var/folders/cl/r3_tlbnx4rj_mp_b2b1gc4f40000gn/T/ipykernel_22061/1854568723.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_demografico_assinantes.rename(columns={
/var/folders/cl/r3_tlbnx4rj_mp_b2b1gc4f40000gn/T/ipykernel_22061/1854568723.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_unified_customers.rename(columns={
/var/folders/cl/r3_tlbnx4rj_mp_b2b1gc4f40000gn/T/ipykernel_22061/1854568723.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [ ]:
# import sqlite3
# import pandas as pd

# # Conectar ao banco de dados SQLite
# db_path = 'Experimentai1.db'
# conn = sqlite3.connect(db_path)
# cursor = conn.cursor()

# # Criação das tabelas no banco de dados
# # DimDemograficoBaseNova
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS DimDemograficoBaseNova (
#         id_demografico INTEGER PRIMARY KEY,
#         id INTEGER,
#         data_nascimento TEXT,
#         genero TEXT,
#         email TEXT,
#         cidade TEXT,
#         estado TEXT,
#         idade INTEGER,
#         faixa_etaria TEXT,
#         bairro TEXT
#     );
# ''')

# # DimGostoBaseNova
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS DimGostoBaseNova (
#         id_gosto INTEGER PRIMARY KEY,
#         bebida BOOLEAN,
#         gastronomia BOOLEAN,
#         moda BOOLEAN,
#         maquiagem BOOLEAN,
#         esporte BOOLEAN,
#         tipo_pet BOOLEAN,
#         pessoa_artistica BOOLEAN,
#         brinquedos_ou_jogos BOOLEAN,
#         tem_pet BOOLEAN,
#         decorar_casa BOOLEAN,
#         alimento_saudavel BOOLEAN,
#         carro BOOLEAN,
#         bebida_alcoolica BOOLEAN,
#         cuidados_pessoais BOOLEAN,
#         tecnologia BOOLEAN,
#         mae_ou_pai BOOLEAN,
#         influencer BOOLEAN,
#         restricoes_alimentares BOOLEAN,
#         objetivo_id INTEGER
#     );
# ''')

# # clienteFato
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS clienteFato (
#         id_fato INTEGER PRIMARY KEY,
#         id_cliente INTEGER,
#         data_inscricao TEXT,
#         rede_social TEXT,
#         renda_id INTEGER,
#         objetivo_id INTEGER,
#         faixa_etaria TEXT,
#         cidade TEXT,
#         estado TEXT,
#         bairro TEXT,
#         id_demografico INTEGER,
#         id_gosto INTEGER,
#         FOREIGN KEY(id_demografico) REFERENCES DimDemograficoBaseNova(id_demografico),
#         FOREIGN KEY(id_gosto) REFERENCES DimGostoBaseNova(id_gosto)
#     );
# ''')

# # DimDemograficoAHT
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS DimDemograficoAHT (
#         id_demografico INTEGER PRIMARY KEY,
#         nome TEXT,
#         email TEXT,
#         idade INTEGER,
#         genero TEXT,
#         renda TEXT
#     );
# ''')

# # DimGostoAHT
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS DimGostoAHT (
#         id_gosto INTEGER PRIMARY KEY,
#         bebida BOOLEAN,
#         gastronomia BOOLEAN,
#         moda BOOLEAN,
#         maquiagem BOOLEAN,
#         esporte BOOLEAN
#     );
# ''')

# # FatoAHT
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS FatoAHT (
#         id_fato_aht INTEGER PRIMARY KEY,
#         id_cliente INTEGER,
#         ja_conhecia BOOLEAN,
#         nota INTEGER,
#         intencao_compra BOOLEAN,
#         onde_gcomprar TEXT,
#         quanto_pagaria REAL,
#         trocar_bebida_por_aht BOOLEAN,
#         recomendar_aht BOOLEAN,
#         id_demografico INTEGER,
#         id_gosto INTEGER,
#         FOREIGN KEY(id_demografico) REFERENCES DimDemograficoAHT(id_demografico),
#         FOREIGN KEY(id_gosto) REFERENCES DimGostoAHT(id_gosto)
#     );
# ''')

# # DimDemograficoPetisco
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS DimDemograficoPetisco (
#         id_demografico INTEGER PRIMARY KEY,
#         email TEXT
#     );
# ''')

# # DimGostoPetisco
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS DimGostoPetisco (
#         id_gosto INTEGER PRIMARY KEY,
#         tipo_pet BOOLEAN
#     );
# ''')

# # FatoPetiscos
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS FatoPetiscos (
#         id_fato_petiscos INTEGER PRIMARY KEY,
#         id_cliente INTEGER,
#         conhecia_produto BOOLEAN,
#         reacao_pet TEXT,
#         tamanho_petisco TEXT,
#         motivo_dar_petisco TEXT,
#         fonte_info_produtos TEXT,
#         atracao_promocao TEXT,
#         avaliacao_custo_beneficio TEXT,
#         local_compra_petiscos TEXT,
#         fator_importante_compra_online TEXT,
#         gasto_mensal_petiscos REAL,
#         disposto_pagar_mais BOOLEAN,
#         motivo_mudanca_fornecedor TEXT,
#         intencao_compra_produto BOOLEAN,
#         frequencia_dar_petisco TEXT,
#         fatores_importantes_escolha TEXT,
#         intencao_proxima_compra TEXT,
#         id_demografico INTEGER,
#         id_gosto INTEGER,
#         FOREIGN KEY(id_demografico) REFERENCES DimDemograficoPetisco(id_demografico),
#         FOREIGN KEY(id_gosto) REFERENCES DimGostoPetisco(id_gosto)
#     );
# ''')

# # Commit para salvar a criação das tabelas
# conn.commit()

# # Função para renomear colunas para corresponder ao esquema do banco
# def rename_columns(df, column_mapping):
#     return df.rename(columns=column_mapping, inplace=False)

# # Renomear e inserir os dados das tabelas
# # DimDemograficoBaseNova
# dim_demografico_base_nova = rename_columns(dim_demografico_base_nova, {
#     'nome': 'id',
#     'sobrenome':'sobrenome',
#     'data_nascimento':'data_nascimento',
#     'genero': 'genero',
#     'email': 'email',
#     'cidade': 'cidade',
#     'estado': 'estado',
#     'idade': 'idade',
#     'faixa_etaria': 'faixa_etaria',
#     'bairro': 'bairro'
# })
# dim_demografico_base_nova['id_demografico'] = range(1, len(dim_demografico_base_nova) + 1)

# # DimGostoBaseNova
# dim_gosto_base_nova = rename_columns(dim_gosto_base_nova, {
#     'bebida': 'bebida',
#     'gastronomia': 'gastronomia',
#     'moda': 'moda',
#     'maquiagem': 'maquiagem',
#     'esporte': 'esporte',
#     'tipo_pet': 'tipo_pet',
#     'pessoa_artistica': 'pessoa_artistica',
#     'brinquedos_ou_jogos': 'brinquedos_ou_jogos',
#     'tem_pet': 'tem_pet',
#     'decorar_casa': 'decorar_casa',
#     'alimento_saudavel': 'alimento_saudavel',
#     'carro': 'carro',
#     'bebida_alcoolica': 'bebida_alcoolica',
#     'cuidados_pessoais': 'cuidados_pessoais',
#     'tecnologia': 'tecnologia',
#     'mae_ou_pai': 'mae_ou_pai',
#     'influencer': 'influencer',
#     'restricoes_alimentares': 'restricoes_alimentares',
#     'objetivo_id': 'objetivo_id'
# })
# dim_gosto_base_nova['id_gosto'] = range(1, len(dim_gosto_base_nova) + 1)

# # Inserção no banco
# def insert_data(df, table_name):
#     df.to_sql(table_name, conn, if_exists='append', index=False)

# # Inserindo os dados
# insert_data(dim_demografico_base_nova, 'DimDemograficoBaseNova')
# insert_data(dim_gosto_base_nova, 'DimGostoBaseNova')

# # Fechar a conexão
# conn.commit()
# conn.close()

# print("Inserção concluída!")


OperationalError: table DimDemograficoBaseNova has no column named sobrenome